<a href="https://colab.research.google.com/github/mauriciodev/spatialgeodesy/blob/main/notebooks/latlon2xyz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1) Dadas as coordenadas: \\
lat = - 08° 03' 3,4697 \\
lon = - 34° 57' 5,4591 \\
h = 20,180 m \\
Converter para cartesianas usando os parâmetros do elipsoide GRS80. \\
2) Implementar o algoritmo iterativo de conversão de coordenadas cartesianas para curvilíneas em uma linguagem de programação qualquer (C++, Matlab, etc.) e aplicar para as coordenadas cartesianas dadas a seguir: \\
X = 5.176.588,653 m \\
Y = -3.618.162,163 m \\
Z = -887.363,920 m \\
3) Aplicar o método direto para converter as coordenadas do item anterior e comparar alatitude obtida no método direto com o método iterativo.

In [ ]:
import numpy as np
def dms2dd(d,m,s):
  if d<0: sign=-1
  else: sign=1
  return sign*(abs(d)+m/60.+s/3600.)
def dd2dms(dd):
  if dd<0: sign=-1
  else: sign=1
  dd=abs(dd)
  d=np.floor(dd)
  dm=(dd-d)*60
  m=np.floor(dm)
  s=(dm-m)*60
  return sign*d,m,s

In [ ]:
inputXYZ=[5176588.653, -3618162.163, -887363.920 ]
inputlat=dms2dd(-8, 3, 3.4697)
inputlon=dms2dd(-34, 57, 5.4591)
inputh=20.180

#GRS80
a=6378137
denf=298.2572221

In [ ]:
  class ellipsoids:
    def __init__(self,a,denf):
      self.a=a
      self.denf=denf
    def b(self):
      return self.a*(1-1/self.denf)
    def e(self):
      e=np.sqrt(1- self.b()**2/(self.a**2))
      return e
    def e2(self):
      e2=np.sqrt(self.a**2/self.b()**2- 1)
      return e2
    def N(self,lat):
      aux=self.e()*np.sin(lat)
      N=self.a/np.sqrt(1-aux*aux)
      return N
    def M(self,lat):
      e=self.e
      M=a*(1-e**2)/(1-e**2 *np.sin(lat)*np.sin(lat))**1.5
      return M
    def xyz2latlon(self,xyz):
      xyz=np.array(inputXYZ)
      lon=np.arctan(xyz[1]/xyz[0]);
      p=np.sqrt(xyz[0]*xyz[0]+xyz[1]*xyz[1]);
      e=self.e();
      e2=self.e2();
      b=self.b();
      u=np.arctan(xyz[2]*a/(p*b));
      lat=np.arctan((xyz[2]+e2*e2*b*np.sin(u)**3)/(p-e*e*a*np.cos(u)**3));
      N0=a/np.sqrt(1-e*e*np.sin(lat)*np.sin(lat));
      h=p/np.cos(lat)-N0;
      return np.array([lon*180/np.pi,lat*180/np.pi, h])
    def xyz2latlonIter(self, xyz):
      lon=np.arctan(xyz[1]/xyz[0]);
      p=np.sqrt(xyz[0]*xyz[0]+xyz[1]*xyz[1]);
      e=self.e();
      error=1;
      h=0; N0=1; lat0=1000;
      while (error>1e-10):
        lat1=np.arctan(xyz[2]/p/(1-e*e*N0/(N0+h)));
        N0=a/np.sqrt(1-e*e*np.sin(lat1)*np.sin(lat1));
        h=p/np.cos(lat1)-N0;
        error=abs(lat1-lat0);
        lat0=lat1;
      return np.array([lon*180/np.pi,lat*180/np.pi, h])
    def latlon2xyz(self,lon,lat,h):
      e=self.e();
      N=self.N(lat);
      x=(N+h)*np.cos(lat)*np.cos(lon);
      y=(N+h)*np.cos(lat)*np.sin(lon);
      z=(N*(1-e*e)+h)*np.sin(lat);
      return np.array([x,y,z])

  grs80=ellipsoids(a,denf)


In [ ]:
lat=inputlat/180*np.pi
lon=inputlon/180*np.pi
xyz=grs80.latlon2xyz(lon,lat,inputh)
print("Transformed coordinates",xyz)
#return res;
print("Expected: X = 5.176.588,653 m ,Y = -3.618.162,163 m, Z = -887.363,920 m")

Transformed coordinates [ 5176588.65344011 -3618162.16304929  -887363.91950406]
Expected: X = 5.176.588,653 m ,Y = -3.618.162,163 m, Z = -887.363,920 m


In [ ]:
xyz

array([ 5176588.65344011, -3618162.16304929,  -887363.91950406])

In [ ]:
res=grs80.xyz2latlon(inputXYZ)
print("XYZ to LonLatH direct: ")
print("lon:", dd2dms(res[0]), res[0])
print("lat:", dd2dms(res[1]), res[1])
print("h:",res[2])

XYZ to LonLatH direct: 
lon: (-34.0, 57.0, 5.459106914982499) -34.951516418587495
lat: (-8.0, 3.0, 3.4697177572337523) -8.05096381048812
h: 20.17968432698399


In [ ]:
grs80=ellipsoids(a,denf)
res=grs80.xyz2latlonIter(inputXYZ)
print("XYZ to LonLatH Iterative: ")
print("lon:", dd2dms(res[0]), res[0])
print("lat:", dd2dms(res[1]), res[1])
print("h:",res[2])


XYZ to LonLatH Iterative: 
lon: (-34.0, 57.0, 5.459106914982499) -34.951516418587495
lat: (-8.0, 3.0, 3.469699999994873) -8.050963805555554
h: 20.17968432698399


# BONUS: Using GDAL

In [ ]:
from gdal import osr
wgsSRS=osr.SpatialReference()
wgsSRS.ImportFromEPSG(4326)
xyzSRS=osr.SpatialReference()
xyzSRS.ImportFromEPSG(4978)


0

In [ ]:
llh2xyz=osr.CoordinateTransformation(wgsSRS,xyzSRS)

In [ ]:
print(llh2xyz.TransformPoint(inputlon,inputlat,inputh))
print(xyz)

(5176588.653438442, -3618162.1630481244, -887363.9195330641)
[ 5176588.65344011 -3618162.16304929  -887363.91950406]


In [ ]:
print(llh2xyz.TransformPoint(inputlon,inputlat,inputh))
print(xyz)

In [ ]:
llh2xyz.reve